<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/cats_dogs_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Last amended: 17th Feb, 2021
# My folder: E:/cats_and_dogs/data
# 
# Data folder:  '/home/ashok/Images/cats_dogs/
# VM: lubuntu_deeplearning_I
# Objectives:
#           i)  Building powerful image classification models using
#               very little data
#           ii) Predicting cats and dogs--Kaggle
#               https://www.kaggle.com/c/dogs-vs-cats
#          iii) Calcualting model weights, stage-by-stage
#

In [ ]:
'''

A. Arrange your data first
==========================

    Download data from: https://www.kaggle.com/c/dogs-vs-cats/data .
    Unzip train.zip and arrange its files as follows:
        data/
            train/
                dogs/                      1000
                    dog001.jpg
                    dog002.jpg
                    ...
                    dog1000.jpg
                cats/                      1000
                    cat001.jpg
                    cat002.jpg
                    ...
                    cat1000.jpg
           validation/
               dogs/                        400
                   dog1001.jpg
                   dog1002.jpg
                   ...
                   dog1400.jpg
              cats/                         400
                  cat1001.jpg
                  cat1002.jpg
                  ...
                  cat1004.jpg

    So we are picking up only 1000 files of each category for training.
    Arrangement of (only) 'training' files in this fashion has an advantage that
    keras automatically knows which images are of cats and which images are
    of dogs. It does automatic labeling of images; we do not have to specify
    explicitly in the code for building training model. This automatic labelling
    is done by ImageGenerator.


In [ ]:
"""
B. Training steps are as follows:
=================================

    1. Arrange training files as above. (Our total samples: nb_train_samples = 2000)
    2. Arrange validation files as above (Valid Samples: nb_validation_samples = 800)
    3. Specify location of all train folder and vaidation folder
    4. Depending upon your backend (tensorflow/theano) decide
       the shape/format of your input image arrays. This is needed in CNN modeling.
    5. Build the CNN model & compile it

    6. Use ImageDataGenerator to augment train images. This is in two steps:
        i)  Create an object with configuration of possible changes in any image
        ii) Use the object to create an iterator with following further configuration:
                a)  Create iterator using flow(), '.flow_from_directory()' method
            In '.flow_from_directory()', specify:
                b)   Where is the directory of your images?
            In ''.flow()' specify X_train, y_train
            Further in both cases:
                b)  Do you also want to resize images, if so, specify these
                c) What batch-size to augment and model at a time;depends upon RAM
                d)  Is classification binary or categorical?

    7. Use ImageDataGenerator to augment validation images. Again the two steps
       as above. But we only resize validation images.
    8. Begin extracting images or training using the iterator fit_generator():
        CNN fit_generator() takes these arguments:
        i)   train-data-iterator (batch-wise source of train images)
        ii)  validation-data generator (batch-wise source of validation images)
        iii) no of epochs


   9. After training has finished, save model weights to a '.h5' file and also
      save model configuration to a json file.


   ------------
   Later, maybe, after some time
   10.Unzip test data file in a folder (within another folder. This is impt.).
   11.Configure test Image Data Generator
   12.Use above configuration and test-folder address, to create a test generator
   13.Load saved cnn model and load network weights in this model from saved h5 file
   14.Use predict() to make predictions on test_generator.
   15.Evaluate predictions

"""

In [ ]:
"""
C. About keras backend:
=======================

    The default keras configuration file is in folder:
        C:\Users\ashokharnal\.keras.  It looks like as below.
        The configurtion is as per the installed backend on your machine:
        tensorflow, theano or CNTK

            {
                    "image_data_format": "channels_last",
                    "epsilon": 1e-07,
                    "floatx": "float32",
                    "backend": "tensorflow"
                    }

            "epsilon" is used instead of zero when division is by zero. 'floatx'
            specifies the datatype that keras will process.
            For 2D data (e.g. image), "channels_last" assumes (rows,cols,channels)
            while "channels_first" assumes (channels,rows,cols)
            (channels stand for RGB colour channels)
"""

In [ ]:
"""
D. Prerequisites:
=================
    Before attempting this problem, pl study Image Augmentation in Moodle at
    http://203.122.28.230/moodle/course/view.php?id=11&sectionid=166#section-9

E. Note
=======
    This is a full code from building model to making predictions for test data.
    AUC is very less as no. of training epochs are just 5. The training consumes
    time but very less memory (around 50%) on an 8GB machine. Vary batch size
    to control memory usage.


F. Make theano as backend
=========================

#    cp /home/ashok/.keras/keras_theano.json  /home/ashok/.keras/keras.json
#    cat /home/ashok/.keras/keras.json
#    source deactivate tensorflow
#    source activate theano
#    ipython
	In file:  ~/.keras/keras.json, set:

	"backend": "theano",    # instead of "tensorflow"

         Also the following environment variable needs to be set in bashrc:

	 export "MKL_THREADING_LAYER=GNU"

'''

In [ ]:
"""

TODO: Use livelossplot
GitHub: https://github.com/stared/livelossplot
SEE example:
https://colab.research.google.com/github/stared/livelossplot/blob/master/examples/keras.ipynb


"""

In [26]:

#%%                                A. Call libraries

#        $ source activate theano
#        $ ipython
# OR in Windows
#       > conda activate tensorflow_env
#       > atom

# 0. Release memory
%reset -f

# 1.0 Data manipulation library
#     Install in 'tf' environment
#     conda install -c anaconda pandas
import pandas as pd

# 1.1 Call libraries for image processing
#     Another preprocessing option is text and sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1.2, Libraries for building sequential CNN model
#      A model is composed of sequence of layered objects
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input

# 1.3.Keras has three backend implementations available: the TensorFlow,
#    the Theano, and CNTK backend.
"""
What is a "backend"?
(http://faroit.com/keras-docs/1.2.0/backend/)
    Keras is a model-level library, providing high-level building blocks
    for developing deep learning models. It does not handle itself low-level
    operations such as tensor products, convolutions and so on. Instead,
    it relies on a specialized, well-optimized tensor manipulation library
    to do so, serving as the "backend engine" of Keras.
    List of low-level functions:
    https://www.tensorflow.org/api_docs/python/tf/keras/backend
"""
from tensorflow.keras import backend as K

# 1.4 Save CNN model configuration
from tensorflow.keras.models import model_from_json

# 1.5 OS related
import os

# 1.6 For ROC plotting
import matplotlib.pyplot as plt

# 1.7
import numpy as np
# conda install scikit-learn
from sklearn import metrics
import time
#from skimage import exposure           # Not used

# 1.8
# conda install -c anaconda pillow
#  Then deactivate and activate environment
#   This step is a must here
from PIL import Image                  # Needed in Windows


In [ ]:
# Make two folders, one for .tar.gz file
#  and the other for unzipped files
! mkdir /root/catsdogs
! mkdir /root/ashok

In [4]:
# Copy tar.gz file from Google drive to Virtual machine
! cp /content/drive/MyDrive/Colab_data_files/cats_dogs.tar.gz  /root/catsdogs
# And check
! ls /root/catsdogs

In [ ]:
# Untar file to folder /root/ashok/
! tar -xvf /root/catsdogs/cats_dogs.tar.gz  -C /root/ashok

In [59]:
# Check folders under 'ashok'
# Folder 'cats_dogs' should exist
! ls /root/ashok/

cats  dogs


In [60]:
#%%                            B. Define constants

# 2. Our constants
# 2.1 Dimensions to which our images will be adjusted
img_width, img_height = 150, 150

# 2.2 Data folder containing all training images, maybe in folders: cats and dogs
#train_data_dir = '/home/ashok/Images/cats_dogs/train'
#train_data_dir ="C:\\Users\\ashok\\Desktop\\chmod\\2. data_augmentation\\cats_dogs\\train"
train_data_dir = '/root/ashok/cats_dogs/train'

# 2.3 What is the total number of training images
#      that should be generated (not what are available)
nb_train_samples = 2000   # Actual: 1000 + 1000 (more) =    2000

# 2.4 Data folder containing all validation images

#validation_data_dir = '/home/ashok/Images/cats_dogs/validation'
#validation_data_dir = "C:\\Users\\ashok\\Desktop\\chmod\\2. data_augmentation\\cats_dogs\\validation"
validation_data_dir = '/root/ashok/cats_dogs/validation'

# 2.5 What is the total no of validation samples that should
#     be generated?
nb_validation_samples = 800   # Actual: 400 + 400 (more) =  800

# Some hyperparameters

# 2.6 Batch size to train at one go:
batch_size = 16             # No of batches = 4000/125 = 32
                            # So per epoch we have 32 batches

# 2.7 How many epochs of training?
epochs = 5                  # For lack of time, let us make it just 5.

# 2.8 No of test samples
test_generator_samples = 300

# 2.9 For test data, what should be batch size
test_batch_size = 25    # This is different from training batch size



In [31]:

# 3. About keras backend
# 3.1 Can get backend configuration values, as:
K.image_data_format()          # Read .keras conf file to findout
K.backend()

# 3.2 What is our backend and input_shape? Decide data shape as per that.
#     Depth goes last in TensorFlow back-end, first in Theano
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:                                         # So, Tensorflow!
    input_shape = (img_width, img_height, 3)


In [37]:

# 4. Create convnet model
#    con->relu->pool->con->relu->pool->con->relu->pool->flatten->fc->fc

#     Call model constructor and then pass on a list of layers
#     https://www.tensorflow.org/api_docs/python/tf/keras/Sequential

model = Sequential()

In [38]:
# 4.1 Add Input layer
model.add(Input(shape =input_shape ))
# 4.2 Add Conv2D layer
model.add(Conv2D(
	             filters=32,                # For every filter there is set of weights
	                                        # For each filter, one bias. So total bias = 32
	             kernel_size=(3, 3),        # For each filter there are 3*3=9 kernel_weights
	             strides = (1,1),           # So output shape will be 148 X 148 (W-F+1).
	                                        # Default strides is 1 only
	             #input_shape=input_shape,   # (150,150,3)
	             use_bias=True,             # Default value is True
	             padding='valid',           # 'valid' => No padding. This is default.
	             name="Ist_conv_layer"
	             )
         )

In [39]:
# 4.3 So what have we done? Can you explain?
#     Total weights = (kernel_weights) * RGB_channel * (filters)  + ToalNoBias
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [40]:
# 4.4 For each neuron in the convolved network,
#     assign an activation function
#     What is relu? See https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/

model.add(Activation('relu'))           # max {0,x}

# 4.5
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [41]:
# 4.6 pool_size:  max pooling window size: (2,2)
#     Default stride for pool-layer is same as pool_size
#     Here: 2 across and 2 down ie (2,2)
#     https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
model.add(MaxPool2D(pool_size=(2, 2)))

# 4.7
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [42]:
# 4.8 Input shape is inferred. Default strides is 1.
#     Note: Activation is specified here only
#     input_shape from top = 74 X 74 X 32
model.add(Conv2D(32,
                (3, 3),
                activation = 'relu',
                name = "IInd_con_layer"))

# 4.9 So how many parameters now?
#     Total weights = (kernel_weights) * (filters_from_earlier_conv) * (filters)  + ToalNoBias
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
IInd_con_layer (Conv2D)      (None, 72, 72, 32)        9248      
Total params: 10,144
Trainable params: 10,144
Non-trainable params: 0
_________________________________________________________________


In [43]:
# model.add(Activation('relu'))

# 4.10 Add another pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))

# 4.11 Summary?
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
IInd_con_layer (Conv2D)      (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
Total params: 10,144
Trainable params: 10,144
Non-trainable params: 0
_________________________________________________________________


In [44]:
# 4.12 Add another conv layer but with 64 filters
#      Total weights = (kernel_weights) * (filters_from_earlier_conv) * (filters)  + ToalNoBias

model.add(Conv2D(64, (3, 3), name = "IIIrd_conv_layer"))

# 4.13
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
IInd_con_layer (Conv2D)      (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
IIIrd_conv_layer (Conv2D)    (None, 34, 34, 64)        18496     
Total params: 28,640
Trainable params: 28,640
Non-trainable params: 0
__________________________________________________

In [45]:
# 4.14
model.add(Activation('relu'))

# 4.15
model.add(MaxPool2D(pool_size=(2, 2)))

# 4.16 Flattens the input. Does not affect the batch size.
#      It merely flattens the earlier layer without adding any weight
#     See summary() next
#     https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten

model.add(Flatten(name = "FlattenedLayer"))

# 4.17
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
IInd_con_layer (Conv2D)      (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
IIIrd_conv_layer (Conv2D)    (None, 34, 34, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 34, 34, 64)       

In [46]:
# 4.18 Dense layer having 64 units
#      dimensionality of the output space.
#      Total weights = hidden_neurons * input_size + bias_foreach_hidden_neurons
#      64 * 18496 + 64
#      Most number of weights come from this layer
#      https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

model.add(Dense(64))

# 4.19
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
IInd_con_layer (Conv2D)      (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
IIIrd_conv_layer (Conv2D)    (None, 34, 34, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 34, 34, 64)       

In [47]:
# 4.20
model.add(Activation('relu'))

# Dropout
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
model.add(Dropout(0.5))

# 4.21
model.summary()


# 4.22 Dense layer having 1 unit
#      dimensionality of the output space.
#      Weights = No of input neurons + bias (64+1)
model.add(Dense(1))

# 4.23
model.summary()

# 4.24
model.add(Activation('sigmoid'))    # tanh vs sigmoid? See Stackoverflow


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Ist_conv_layer (Conv2D)      (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
IInd_con_layer (Conv2D)      (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
IIIrd_conv_layer (Conv2D)    (None, 34, 34, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 34, 34, 64)       

In [48]:
# 4.25 Compile model
model.compile(
              loss='binary_crossentropy',  # Metrics to be adopted by convergence-routine
              optimizer='rmsprop',         # Strategy for convergence?
              metrics=['accuracy'])        # Metrics, I am interested in




In [49]:

#%%                            D. Create Data generators


## 5. Image augmentation
# 5.1 Define a preprocessing function
def preprocess(img):
	# Histogram equalization
	# WITHOUT IT RESULTS ARE BETTER
	# http://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_equalize.html
    # img_eq = exposure.equalize_hist(img)
    # Do something more with image
    return img


In [50]:
# 5.2 Config1: Augmentation configuration for training samples
#     Instantiate ImageDataGenerator object with requisite configuration
#     https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

tr_dtgen = ImageDataGenerator(
                              rescale=1. / 255,      # Normalize colour intensities in 0-1 range
                              shear_range=0.2,       # Shear varies from 0-0.2
                              zoom_range=0.2,
                              horizontal_flip=True,
                              preprocessing_function=preprocess
                              )

In [51]:

# 5.3 Config2: Create iterator from 'train_datagen'.
#     We use flow() or flow_from_directory() methods to further
#     configure and return an iterator object.
#     See at the end of code: Differences between flow()
#     and flow_from_directory
#     Pl see: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory

train_generator = tr_dtgen.flow_from_directory(
                                               train_data_dir,       # Data folder of cats & dogs
                                               target_size=(img_width, img_height),  # Resize images
                                               batch_size=batch_size,  # Return images in batches
                                               class_mode='binary'   # Output labels will be 1D binary labels
                                                                     # [[1],[0],[0],[1]]
                                                                     # If 'categorical' output labels will be
                                                                     # 2D OneHotEncoded: [[1,0],[0,1],[0,1],[1,0]]
                                                                     # If 'binary' use 'sigmoid' at output
                                                                     # If 'categorical' use softmax at output

                                                )


Found 2000 images belonging to 2 classes.


In [52]:

# 5.4 Augmentation configuration we will use
#     for validation. Only rescaling of pixels

val_dtgen = ImageDataGenerator(rescale=1. / 255)

In [61]:

# 5.4.2 validation data
validation_generator = val_dtgen.flow_from_directory(
                                                     validation_data_dir,
                                                     target_size=(img_width, img_height),   # Resize images
                                                     batch_size=batch_size,    # batch size to augment at a time
                                                     class_mode='binary'  # Return 1D array of class labels
                                                     )

Found 800 images belonging to 2 classes.


In [54]:

#%%                           E. Fit model & save CNN network weights


## 6. Model fitting

# 6.1 Manual process of fitting. Get infinite images
#     Can experiment with infinite images. We will
#     generate upto 3200 images
#     https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit

start = time.time()   # 6 minutes
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in train_generator:
        model.fit(x_batch, y_batch)
        batches += 1
        print ("Epoch: {0} , Batches: {1}".format(e,batches))
        if batches > 200:    # 200 * 16 = 3200 images
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

end = time.time()
(end - start)/60


Epoch 0
1/1 [==============================] - 2s 2s/step - loss: 0.6907 - accuracy: 0.4375
Epoch: 0 , Batches: 1
1/1 [==============================] - 0s 410ms/step - loss: 2.6257 - accuracy: 0.5000
Epoch: 0 , Batches: 2
1/1 [==============================] - 0s 421ms/step - loss: 0.8124 - accuracy: 0.3750
Epoch: 0 , Batches: 3
1/1 [==============================] - 0s 407ms/step - loss: 0.7068 - accuracy: 0.5000
Epoch: 0 , Batches: 4
1/1 [==============================] - 0s 403ms/step - loss: 0.6887 - accuracy: 0.4375
Epoch: 0 , Batches: 5
1/1 [==============================] - 0s 407ms/step - loss: 0.7210 - accuracy: 0.4375
Epoch: 0 , Batches: 6
1/1 [==============================] - 0s 401ms/step - loss: 0.6954 - accuracy: 0.5625
Epoch: 0 , Batches: 7
1/1 [==============================] - 0s 405ms/step - loss: 0.7255 - accuracy: 0.5625
Epoch: 0 , Batches: 8
1/1 [==============================] - 0s 408ms/step - loss: 0.7060 - accuracy: 0.3750
Epoch: 0 , Batches: 9
1/1 [=========

9.98195558389028

In [62]:
# 6.2 fit_generator() directly pulls data from iterators
#     But number of images cannot exceed those available
#     Ref: https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit_generator

start = time.time()
history = model.fit_generator(
                              generator = train_generator,          # First argument is always training data generator
                              steps_per_epoch=nb_train_samples // batch_size, # How many batches per epoch?
                                                                              # Can be any number as generator loops indefinitely
                              epochs=epochs,                        # No of epochs
                              validation_data=validation_generator, # Get validation data from validation generator
                              verbose = 1,                          # Do not be silent
                              validation_steps=nb_validation_samples // batch_size
                              )

end = time.time()
(end - start)/60


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
125/125 [==============================] - 73s 578ms/step - loss: 0.5473 - accuracy: 0.7465 - val_loss: 0.6819 - val_accuracy: 0.7013
Epoch 2/5
125/125 [==============================] - 72s 575ms/step - loss: 0.5320 - accuracy: 0.7340 - val_loss: 0.6004 - val_accuracy: 0.6687
Epoch 3/5
125/125 [==============================] - 72s 576ms/step - loss: 0.5087 - accuracy: 0.7660 - val_loss: 0.5623 - val_accuracy: 0.7337
Epoch 4/5
125/125 [==============================] - 72s 575ms/step - loss: 0.5229 - accuracy: 0.7470 - val_loss: 0.5465 - val_accuracy: 0.7450
Epoch 5/5
125/125 [==============================] - 72s 575ms/step - loss: 0.5245 - accuracy: 0.7575 - val_loss: 0.4918 - val_accuracy: 0.7663


6.017813805739085

In [63]:
# 7.0 Model evaluation

# 7.1 Using generator
#     https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate
result = model.evaluate(validation_generator,
                                  verbose = 1,
                                  steps = 4        # How many batches
                                  )


# 7.1.1
result     # ['loss', 'accuracy']

4/4 [==============================] - 1s 148ms/step - loss: 0.5154 - accuracy: 0.7812


[0.5153961181640625, 0.78125]

In [ ]:

# 8.0 Make predictions

# 8.1 Using generator
#     https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict
pred = model.predict(validation_generator, steps = 2)

# 8.1.1
pred


In [ ]:

# 8.2 Manually
pred = []
steps = 0
start = time.time()
for x_batch, y_batch in validation_generator:
        pred.append(model.predict(x_batch))
        steps += 1
        if steps >= 2:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

end = time.time()
(end - start)/60

# 8.2.1
pred
